# [176. Second Highest Salary](https://leetcode.com/problems/second-highest-salary/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Employee

<pre>+-------------+------+
| Column Name | Type |
+-------------+------+
| id          | int  |
| salary      | int  |
+-------------+------+</pre>
id is the primary key (column with unique values) for this table.
Each row of this table contains information about the salary of an employee.
 

Write a solution to find the second highest salary from the Employee table. If there is no second highest salary, return null (return None in Pandas).

The result format is in the following example.

 

Example 1:

Input: 
Employee table:
<pre>+----+--------+
| id | salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+</pre>
Output: 
<pre>+---------------------+
| SecondHighestSalary |
+---------------------+
| 200                 |
+---------------------+</pre>
Example 2:

Input: 
Employee table:
<pre>+----+--------+
| id | salary |
+----+--------+
| 1  | 100    |
+----+--------+</pre>
Output: 
<pre>+---------------------+
| SecondHighestSalary |
+---------------------+
| null                |
+---------------------+</pre>

In [1]:
# pandas schema

import pandas as pd

#data = [[1, 100], [2, 200], [3, 300]] # example 1
data = [[1, 100]]  # example 2
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id': 'int64', 'salary': 'int64'})

In [2]:
# to spark dataframe

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

employee_df = spark.createDataFrame(employee)
employee_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/11 20:57:31 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/11 20:57:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/11 20:57:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+------+
|id |salary|
+---+------+
|1  |100   |
+---+------+


In [3]:
from pyspark.sql.types import StructType, StructField, StringType, Row
from pyspark.sql import Window

# solving in spark dataframe
employee_df.select(F.col('salary')).distinct() \
    .unionAll(spark.createDataFrame([Row(col1=None)], schema=StructType([StructField("salary", StringType(), True)]))) \
    .withColumn('rn', F.row_number().over(Window.orderBy(F.col('salary').desc()))) \
    .filter(F.col('rn')==2)\
    .select(F.col('salary').alias('SecondHighestSalary'))\
    .show()

23/11/11 20:57:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 20:57:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 20:57:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 20:57:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 20:57:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 20:57:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 2

+-------------------+
|SecondHighestSalary|
+-------------------+
|               null|
+-------------------+


In [4]:
# solving in spark SQL

employee_df.createOrReplaceTempView('employee')

spark.sql('''
select salary as SecondHighestSalary
from (
    select salary, row_number() over (order by salary desc) as rn from 
    (select distinct salary from employee union all select null) n
) s where s.rn =2;
''').show()

23/11/11 20:57:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 20:57:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 20:57:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 20:57:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 20:57:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 20:57:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 2

+-------------------+
|SecondHighestSalary|
+-------------------+
|               null|
+-------------------+


In [5]:
spark.stop()